# Caracterização de eletrofáceis por meio de inteligência artificial com abordagem supervisionada
## Reclassificação da litologia metamorfica em xisto ou filito

- **Nome do Arquivo**: tratamento_dados_metamorfica.ipynb
- **Autor**: Rafael Takeguma Goto
- **Versão**: 1.0
- **Data de Criação**: 11/07/2024
- **Data de Modificação**: 12/07/2024

### Descrição: 
Este notebook contém a etapa do trabalho referente à reclassificação da litologia METAMORFICA NAO IDE em XISTO ou FILITO.

Para reclassificar a metamorfica, será utilizado um modelo de classificação de floresta aleatória, que utilizará as propriedades de perfis dos registros de xisto e filito para aprender e ser capaz de rotular a metamorfica como uma dessas duas litologias.

### Leitura do conjunto de dados

O conjunto de dados é constituído por um arquivo csv, que é resultado da concatenação dos arquivos csv gerados a partir da fusão da extração dos arquivos dlis com os arquivos agp de cada poço, seguida da aplicação dos filtros de constantes, nulos e perfis (DCAL e DRHO).

In [1]:
import pandas as pd

path_pocos_pre_transicao = 'Dados-filtrados-concatenados\pocos_pre_transicao.csv'

dados_concatenados_leitura = pd.read_csv(path_pocos_pre_transicao)

dados_concatenados = pd.DataFrame(dados_concatenados_leitura)

### Separa os registros de XISTO e FILITO
Separa as linhas (registros) em que a litologia é XISTO ou FILITO para treinar o modelo com base nesses dados.

In [2]:
dados_xisto_filito = dados_concatenados[(dados_concatenados['Litologia'] == 'XISTO') | (dados_concatenados['Litologia'] == 'FILITO')]

### Divide os dados em treinamento e teste
Divide os dados armazenados em dados_xisto_filito em 2/3 para treinamento do modelo, e 1/3 para teste do modelo.

In [3]:
from sklearn.model_selection import train_test_split

# Define as variáveis que serão utilizadas (propriedades de perfis) 
propriedades = ['GR', 'DCAL', 'RESD', 'DT', 'RHOB', 'DRHO', 'NPHI', 'PE']

# Separa as variáveis (propriedades de perfis) e armazena em X
X = dados_xisto_filito[propriedades]

# Separa a variável alvo (litologia) e armazena em y
y = dados_xisto_filito['Litologia']

# Divide os dados em 2/3 para treinamento e 1/3 para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

### Treinamento e teste do modelo
Utiliza os dados de treinamento armazenados em X_train e y_train para treinar o modelo de classificação. De forma semelhante, utiliza os dados de teste armazenados em X_test e y_test para testar a acurácia do modelo.

In [4]:
from sklearn.ensemble import RandomForestClassifier

# Treina o modelo
modelo = RandomForestClassifier()
modelo.fit(X_train, y_train)

# Testa o modelo
acuracia = modelo.score(X_test, y_test)
print(f"Acurácia do modelo: {acuracia:2f}")

Acurácia do modelo: 0.991935


### Reclassificação dos registros de metamorfica
Utiliza o modelo treinado acima para prever se os registros que possuem METAMORFICA NAO IDE como litologia tem afinidade para XISTO ou FILITO.

In [5]:
# Separa as linhas (registros) em que a litologia é METAMORFICA NAO IDE 
dados_metamorfica = dados_concatenados[dados_concatenados['Litologia'] == 'METAMORFICA NAO IDE']

# Separa as linhas (registros) da litologia METAMORFICA NAO IDE apenas com as variáveis preditivas
novas_linhas = dados_metamorfica[X.columns]

# Realiza as predições das novas linhas para xisto ou filito
predicoes = modelo.predict(novas_linhas)

# Reclassifica os registros de METAMORFICA NAO IDE em dados_concatenados para XISTO ou FILITO com as predições 
dados_concatenados.loc[dados_concatenados['Litologia'] == 'METAMORFICA NAO IDE', 'Litologia'] = predicoes

In [21]:
dados_concatenados[dados_concatenados['Litologia'] == 'FILITO']

,Poço,Profundidade,UnidadeF,UnidadeM,UnidadeC,TopoUnidade,BaseUnidade,Topo,Base,Litologia,...,DCAL,GR,RESD,DT,RHOB,DRHO,NPHI,PE,interpolada,PE.1
3954,1-BRSA-459-SE,521.1,EMBASA,EMBASA,NaN,False,False,False,False,FILITO,...,0.089493,134.055191,243.180344,60.591904,2.818947,0.006032,0.102083,3.600137,False,NaN
4133,1-BRSA-459-SE,539.0,EMBASA,EMBASA,NaN,False,False,False,False,FILITO,...,0.099505,142.522018,232.958694,61.127762,2.825869,0.004325,0.118498,3.952035,False,NaN
4134,1-BRSA-459-SE,539.1,EMBASA,EMBASA,NaN,False,False,False,False,FILITO,...,0.099505,142.127342,240.881569,60.667473,2.819057,-0.000348,0.118569,3.952022,True,NaN
4135,1-BRSA-459-SE,539.2,EMBASA,EMBASA,NaN,False,False,False,False,FILITO,...,0.099505,141.732666,248.804443,60.207184,2.812245,-0.005021,0.118640,3.952009,False,NaN
4136,1-BRSA-459-SE,539.3,EMBASA,EMBASA,NaN,False,False,False,False,FILITO,...,0.106181,142.245758,245.066879,60.079948,2.812485,-0.004553,0.115976,3.929725,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29541,1-BRSA-659-SE,725.6,EMBASA,EMBASA,NaN,False,False,False,False,FILITO,...,0.557294,144.340866,206.596893,62.775513,2.754881,0.079730,0.154467,3.567569,False,NaN
29542,1-BRSA-659-SE,725.7,EMBASA,EMBASA,NaN,False,False,False,False,FILITO,...,0.524774,142.978241,191.050293,63.628063,2.743806,0.077803,0.158077,3.558456,False,NaN
29543,1-BRSA-659-SE,725.8,EMBASA,EMBASA,NaN,False,False,False,False,FILITO,...,0.532904,142.008179,196.273911,64.222450,2.721323,0.066145,0.159886,3.531438,False,NaN
29544,1-BRSA-659-SE,725.9,EMBASA,EMBASA,NaN,False,False,False,False,FILITO,...,0.557294,141.163284,202.735947,64.530106,2.703651,0.057787,0.161343,3.543736,False,NaN


In [20]:
import numpy as np

# Contagem de ocorrências de cada elemento único
elementos_unicos, contagem = np.unique(predicoes, return_counts=True)

# Exibindo os resultados
for elemento, qtd in zip(elementos_unicos, contagem):
    print(f"{elemento}: {qtd} registro(s)")

FILITO: 20 registro(s)
XISTO: 565 registro(s)
